In [1]:
# from google.colab import drive
import pandas as pd
import numpy as np
import math
import string
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, f1_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import PassiveAggressiveClassifier

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import gensim
from gensim.models import Word2Vec

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as VS

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\udit1\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\udit1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# drive.mount('/content/drive')

In [4]:
datapath = "H1_train.csv"
trainset = pd.read_csv(datapath);

#remove contractions
def decontract(text):
    text = text.replace(r"can't", "can not ")
    text = text.replace(r"n't", " not ")
    text = text.replace(r"'re", " are ")
    text = text.replace(r"'s ", " is ")
    text = text.replace(r"'d ", " would ")
    text = text.replace(r"'ll", " will ")
    text = text.replace(r"'t", " not ")
    text = text.replace(r"'ve", " have ")
    text = text.replace(r"'m", " am ")
    return text
    
    
def text_preprocessing(df):
    '''
    input : df is the dataframe(obtained by  pd.read_csv(H1_Offensive_Language_Identification_train.csv) ) 
    output: None . Tweets in df will be preprocessed internally.
    '''
    punctuations = string.punctuation
    
    df.loc[:, 'tweet'] = df.tweet.str.replace('@USER', '') #Remove mentions (@USER)
    df.loc[:, 'tweet'] = df.tweet.str.replace('URL', '') #Remove URLs
    df.loc[:, 'tweet'] = df.tweet.str.replace('&amp', 'and') #Replace ampersand (&) with and
    df.loc[:, 'tweet'] = df.tweet.str.replace('&lt','') #Remove &lt
    df.loc[:, 'tweet'] = df.tweet.str.replace('&gt','') #Remove &gt
    df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
    df.loc[:, 'tweet'] = df.tweet.str.replace('’',"'") #convert webpage apostrophe to ascii 39 
    df.loc[:, 'tweet'] = df.tweet.str.replace('‘',"'") #convert webpage apostrophe to ascii 39 
    df.loc[:, 'tweet'] = df.tweet.str.lower() #Lowercase
    for index_label, row_series in df.iterrows():
    
      df.at[index_label , 'tweet'] = decontract(row_series['tweet'])  
    print(df.loc[11,'tweet'])
    #Remove punctuations
    for punctuation in punctuations:
        df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')
    
    #Remove emojis
    df.loc[:, 'tweet'] = df.astype(str).apply(
        lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii')
    )

    #remove multiple spaces
    for index_label, row_series in df.iterrows():
    
      df.at[index_label , 'tweet'] = " ".join(row_series['tweet'].split(" ")) 
    df.loc[:, 'tweet'] = df.tweet.str.strip() #Trim leading and trailing whitespaces

text_preprocessing(trainset)

C:\Users\udit1\AppData\Local\Temp\ipykernel_17912\2185684462.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers


 what is the difference between #kavanaugh and    one of these men admitted to groping a  year old girl years ago.  the other is going to be #confirmed to the scj   #demsarefrauds #demsaredone   #walkawaydemocrats #redwave #voteredsaveamerica #trumptrain #maga 


C:\Users\udit1\AppData\Local\Temp\ipykernel_17912\2185684462.py:40: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')


In [5]:
trainset.head()

,tweet,label,id
0,she should ask a few native americans what the...,OFF,0
1,go home you are drunk maga trump,OFF,1
2,amazon is investigating chinese employees who ...,NOT,2
3,someone should have taken this piece of shit t...,OFF,3
4,obama wanted liberals and illegals to move int...,NOT,4


In [6]:
trainset.shape

(13240, 3)

In [8]:
np.sum(trainset["label"] == "OFF")

4400

In [9]:
np.sum(trainset["label"] == "NOT")

8840

Observation 1 : The dataset is not balanced

In [10]:
X_train, X_val, y_train, y_val = train_test_split(trainset["tweet"], trainset["label"], test_size=0.33, random_state=42)

In [11]:
len(X_train)

8870

In [12]:
X_train, X_val, y_train, y_val=list(X_train), list(X_val), list(y_train), list(y_val)

In [13]:
len(X_val)

4370

In [14]:
len(y_train)

8870

In [15]:
!pip install nltk
!pip install sentence-transformers
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [16]:
X_train=np.array(X_train)
X_val=np.array(X_val)

y_train=np.array(y_train)
y_val=np.array(y_val)

In [17]:
X_train

array(['i believe that serena would not cheat yet even her manager has admitted to breaking the rules this was not her finest hour first a temper tantrum then calling the umpire a thief that crossed the line she is the one that should apologize',
       'is not  this bodyshaming  good to see the feminists and liberals have no issue with bodyshaming as long as it is directed towards a man',
       'funny how he believes antifa is a thing but calls himself reality checks',
       ..., 'you are so right',
       'sam is act is working i swear she is acting', 'chuckle heads'],
      dtype='<U287')

In [18]:

import re
from nltk.corpus import stopwords

from sklearn import linear_model
from  sentence_transformers import SentenceTransformer

from sklearn import svm
import torch 

from scipy.stats import spearmanr

from sentence_transformers import InputExample, losses, models
from sentence_transformers import util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
from torch import nn
from datetime import datetime
import math

In [19]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)

In [20]:
X_val_tfidf=vectorizer.transform(X_val)

In [21]:
print(X_train_tfidf.shape)
print(X_val_tfidf.shape)

(8870, 15767)
(4370, 15767)


In [22]:
dict_models={}
dict_accuracy={}

In [23]:
trainset_tfidf=vectorizer.transform(list(trainset["tweet"]))

In [26]:
print(X_train_tfidf.shape)
print(X_val_tfidf.shape)
print(trainset_tfidf.shape)

(8870, 15767)
(4370, 15767)
(13240, 15767)


Models Using TFIDF

In [29]:
dict_models_tfidf={}
dict_accuracy_report_tfidf={}

x_train=X_train_tfidf
x_test=X_val_tfidf
y_test=y_val

In [30]:
print(x_train.shape)
print(x_test.shape)
print(np.array(y_train).shape)
print(np.array(y_test).shape)

(8870, 15767)
(4370, 15767)
(8870,)
(4370,)


In [31]:
logictic_model = LogisticRegression().fit(x_train,y_train)
y_preds = logictic_model.predict(x_test)
report = classification_report( y_test, y_preds )
print(report)
acc=accuracy_score(y_test,y_preds)
print("Logistic Regression, Accuracy Score:" , acc)
dict_models_tfidf["logictic_model"]=logictic_model
dict_accuracy_report_tfidf["Logistic Regression"]=report

              precision    recall  f1-score   support

         NOT       0.74      0.96      0.84      2903
         OFF       0.80      0.35      0.48      1467

    accuracy                           0.75      4370
   macro avg       0.77      0.65      0.66      4370
weighted avg       0.76      0.75      0.72      4370

Logistic Regression, Accuracy Score: 0.7517162471395881


In [34]:
y_train_pred=logictic_model.predict(x_train)
report = classification_report( y_train, y_train_pred )
print(report)

              precision    recall  f1-score   support

         NOT       0.78      0.98      0.87      5937
         OFF       0.93      0.45      0.61      2933

    accuracy                           0.81      8870
   macro avg       0.86      0.72      0.74      8870
weighted avg       0.83      0.81      0.79      8870



In [35]:
dict_accuracy_report_tfidf

{'Logistic Regression': '              precision    recall  f1-score   support\n\n         NOT       0.74      0.96      0.84      2903\n         OFF       0.80      0.35      0.48      1467\n\n    accuracy                           0.75      4370\n   macro avg       0.77      0.65      0.66      4370\nweighted avg       0.76      0.75      0.72      4370\n'}

In [36]:
classifier_mnb=MultinomialNB()
classifier_mnb.fit(x_train, y_train)
pred = classifier_mnb.predict(x_test)
score = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
print()
print("accuracy_matrix")
print(confusion_matrix(y_test,pred))
print()
report=classification_report( y_test, pred )
print(f"Classification Report : \n\n{classification_report(y_test, pred)}")
dict_models_tfidf["MultinomialNB"]=classifier_mnb
dict_accuracy_report_tfidf["MultinomialNB"]=report

accuracy:   0.694

accuracy_matrix
[[2879   24]
 [1315  152]]

Classification Report : 

              precision    recall  f1-score   support

         NOT       0.69      0.99      0.81      2903
         OFF       0.86      0.10      0.19      1467

    accuracy                           0.69      4370
   macro avg       0.78      0.55      0.50      4370
weighted avg       0.75      0.69      0.60      4370



In [37]:
y_train_pred=classifier_mnb.predict(x_train)
report = classification_report( y_train, y_train_pred )
print(report)

              precision    recall  f1-score   support

         NOT       0.72      1.00      0.84      5937
         OFF       0.97      0.23      0.38      2933

    accuracy                           0.74      8870
   macro avg       0.85      0.62      0.61      8870
weighted avg       0.81      0.74      0.69      8870



In [40]:
## tuning with different values of k, also know as add k smoothing
pre_score=0
best_k=0
mnb_tuned_classfier=MultinomialNB()
for a in np.arange(0, 2, 0.1):
  classifier=MultinomialNB(alpha=a)
  classifier.fit(x_train,y_train)
  pred = classifier.predict(x_test)
  score = accuracy_score(y_test, pred)
  if(score>pre_score):
    mnb_tuned_classifier=classifier
    pre_score=score
    best_k=a

mnb_tuned_classifier.fit(x_train,y_train)
pred = mnb_tuned_classifier.predict(x_test)
score = accuracy_score(y_test, pred)
print("accuracy: %0.3f" %score)
print("best_k: %0.3f" % best_k)
print()
print("accuracy_matrix")
report=confusion_matrix(y_test,pred)
print(confusion_matrix(y_test,pred))
print()
print(f"Classification Report : \n\n{classification_report(y_test, pred)}")
dict_models["Tuned_MultinomialNB"]=mnb_tuned_classifier
dict_accuracy[score]="Tuned_MultinomialNB"

dict_models_tfidf["Tuned_MultinomialNB"]=mnb_tuned_classifier
dict_accuracy_report_tfidf["Tuned_MultinomialNB"]=report

C:\Users\udit1\AppData\Roaming\Python\Python310\site-packages\sklearn\naive_bayes.py:591: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(


accuracy: 0.720
best_k: 0.200

accuracy_matrix
[[2703  200]
 [1025  442]]

Classification Report : 

              precision    recall  f1-score   support

         NOT       0.73      0.93      0.82      2903
         OFF       0.69      0.30      0.42      1467

    accuracy                           0.72      4370
   macro avg       0.71      0.62      0.62      4370
weighted avg       0.71      0.72      0.68      4370



In [41]:
y_train_pred=mnb_tuned_classifier.predict(x_train)
report = classification_report( y_train, y_train_pred )
print(report)

              precision    recall  f1-score   support

         NOT       0.89      0.99      0.94      5937
         OFF       0.98      0.75      0.85      2933

    accuracy                           0.91      8870
   macro avg       0.93      0.87      0.89      8870
weighted avg       0.92      0.91      0.91      8870



In [42]:
model_pa = PassiveAggressiveClassifier(C = 0.6, random_state = 5)
model_pa.fit(x_train, y_train)
test_pred = model_pa.predict(x_test)
score = accuracy_score(y_test, test_pred)
print(f"Tuned Test Set Accuracy : {accuracy_score(y_test, test_pred) * 100} %\n\n")  
print()
print("accuracy_matrix")
print(confusion_matrix(y_test,test_pred))
print()
report=classification_report(y_test, test_pred)
print(f"Classification Report : \n\n{classification_report(y_test, test_pred)}")
dict_models["passive_aggresive"]=model_pa
dict_accuracy[score]="passive_aggresive"

dict_models_tfidf["passive_aggresive"]=model_pa
dict_accuracy_report_tfidf["passive_aggresive"]=report

Tuned Test Set Accuracy : 70.48054919908466 %



accuracy_matrix
[[2298  605]
 [ 685  782]]

Classification Report : 

              precision    recall  f1-score   support

         NOT       0.77      0.79      0.78      2903
         OFF       0.56      0.53      0.55      1467

    accuracy                           0.70      4370
   macro avg       0.67      0.66      0.66      4370
weighted avg       0.70      0.70      0.70      4370



In [43]:
y_train_pred=model_pa.predict(x_train)
report = classification_report( y_train, y_train_pred )
print(report)

              precision    recall  f1-score   support

         NOT       0.99      1.00      1.00      5937
         OFF       0.99      0.99      0.99      2933

    accuracy                           0.99      8870
   macro avg       0.99      0.99      0.99      8870
weighted avg       0.99      0.99      0.99      8870



In [45]:
#model-3
#decission tree

dt_classifier=DecisionTreeClassifier()
dt_classifier.fit(x_train,y_train)
test_pred = dt_classifier.predict(x_test)
score=accuracy_score(y_test, test_pred)
print(f"Test Set Accuracy : {accuracy_score(y_test, test_pred) * 100} %\n\n")
print()
print("accuracy_matrix")
print(confusion_matrix(y_test,test_pred))
print()
report=classification_report(y_test, test_pred)
print(f"Classification Report : \n\n{classification_report(y_test, test_pred)}")
  
dict_models["DecisionTree"]=dt_classifier
dict_accuracy[score]="DecisionTree"

dict_models_tfidf["DecisionTree"]=dt_classifier
dict_accuracy_report_tfidf["DecisionTree"]=report

Test Set Accuracy : 70.64073226544623 %



accuracy_matrix
[[2337  566]
 [ 717  750]]

Classification Report : 

              precision    recall  f1-score   support

         NOT       0.77      0.81      0.78      2903
         OFF       0.57      0.51      0.54      1467

    accuracy                           0.71      4370
   macro avg       0.67      0.66      0.66      4370
weighted avg       0.70      0.71      0.70      4370



In [46]:
y_train_pred=dt_classifier.predict(x_train)
report = classification_report( y_train, y_train_pred )
print(report)

              precision    recall  f1-score   support

         NOT       1.00      1.00      1.00      5937
         OFF       1.00      0.99      1.00      2933

    accuracy                           1.00      8870
   macro avg       1.00      1.00      1.00      8870
weighted avg       1.00      1.00      1.00      8870



In [47]:
clf_random= RandomForestClassifier()
clf_random.fit(x_train, y_train)
pred = clf_random.predict(x_test)
score = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
print()
print("accuracy_matrix")
print(confusion_matrix(y_test,pred))
print()
report=classification_report(y_test, pred)
print(f"Classification Report : \n\n{classification_report(y_test, pred)}")

dict_models["RandomForestClassifier"]=clf_random
dict_accuracy[score]="RandomForestClassifier"

dict_models_tfidf["RandomForestClassifier"]=clf_random
dict_accuracy_report_tfidf["RandomForestClassifier"]=report

accuracy:   0.755

accuracy_matrix
[[2802  101]
 [ 971  496]]

Classification Report : 

              precision    recall  f1-score   support

         NOT       0.74      0.97      0.84      2903
         OFF       0.83      0.34      0.48      1467

    accuracy                           0.75      4370
   macro avg       0.79      0.65      0.66      4370
weighted avg       0.77      0.75      0.72      4370



In [48]:
y_train_pred=clf_random.predict(x_train)
report = classification_report( y_train, y_train_pred )
print(report)

              precision    recall  f1-score   support

         NOT       1.00      1.00      1.00      5937
         OFF       1.00      1.00      1.00      2933

    accuracy                           1.00      8870
   macro avg       1.00      1.00      1.00      8870
weighted avg       1.00      1.00      1.00      8870



In [49]:
# tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),max_df=0.75, min_df=5, max_features=10000)

# # TF-IDF feature matrix
# tfidf = tfidf_vectorizer.fit_transform(tweets )
# tfidf

In [50]:
dict_models_tfidf

{'logictic_model': LogisticRegression(),
 'MultinomialNB': MultinomialNB(),
 'Tuned_MultinomialNB': MultinomialNB(alpha=0.2),
 'passive_aggresive': PassiveAggressiveClassifier(C=0.6, random_state=5),
 'DecisionTree': DecisionTreeClassifier(),
 'RandomForestClassifier': RandomForestClassifier()}

In [51]:
dict_accuracy_report_tfidf

{'Logistic Regression': '              precision    recall  f1-score   support\n\n         NOT       0.74      0.96      0.84      2903\n         OFF       0.80      0.35      0.48      1467\n\n    accuracy                           0.75      4370\n   macro avg       0.77      0.65      0.66      4370\nweighted avg       0.76      0.75      0.72      4370\n',
 'MultinomialNB': '              precision    recall  f1-score   support\n\n         NOT       0.69      0.99      0.81      2903\n         OFF       0.86      0.10      0.19      1467\n\n    accuracy                           0.69      4370\n   macro avg       0.78      0.55      0.50      4370\nweighted avg       0.75      0.69      0.60      4370\n',
 'Tuned_MultinomialNB': array([[2703,  200],
        [1025,  442]], dtype=int64),
 'passive_aggresive': '              precision    recall  f1-score   support\n\n         NOT       0.77      0.79      0.78      2903\n         OFF       0.56      0.53      0.55      1467\n\n    accu

In [52]:
num_models=len(dict_accuracy_report_tfidf)

In [53]:
models_list=['logictic_model', 'MultinomialNB', 'Tuned_MultinomialNB', 'passive_aggresive', 'DecisionTree', 'RandomForestClassifier']
models_list=['logictic_model', 'MultinomialNB', 'passive_aggresive', 'DecisionTree', 'RandomForestClassifier']

In [64]:
dict_outputs={"train":{}, "val": {}}

In [65]:
for model in models_list:
    dict_outputs["val"][model]=dict_models_tfidf[model].predict(x_test)
    dict_outputs["train"][model]=dict_models_tfidf[model].predict(x_train)

In [66]:
dict_outputs["val"]["logictic_model"].shape

(4370,)

In [71]:
list_train_preds=np.array(list(dict_outputs["train"].values()))

In [68]:
list_val_preds=np.array(list(dict_outputs["val"].values()))

In [69]:
def majority_prediction(list_predictions):
  final_prediction=[]

  for i in range(len(list_predictions[0])):
    count_off=0
    count_not=0

    for j in range(len(list_predictions)):
      if list_predictions[j][i]=="OFF":
        count_off+=1
      else:
        count_not+=1
    if count_off>count_not:
      final_prediction.append("OFF")
    else:
      final_prediction.append("NOT")

  return np.array(final_prediction)


In [72]:
majority_val_prediction=majority_prediction(list_val_preds)
majority_train_prediction=majority_prediction(list_train_preds)

In [73]:
print(majority_val_prediction.shape)

print(majority_train_prediction.shape)

(4370,)
(8870,)


In [74]:
print(classification_report(y_test, majority_val_prediction))

              precision    recall  f1-score   support

         NOT       0.74      0.96      0.84      2903
         OFF       0.83      0.33      0.48      1467

    accuracy                           0.75      4370
   macro avg       0.78      0.65      0.66      4370
weighted avg       0.77      0.75      0.72      4370



In [76]:
print(classification_report(y_train, majority_train_prediction))

              precision    recall  f1-score   support

         NOT       1.00      1.00      1.00      5937
         OFF       1.00      0.99      0.99      2933

    accuracy                           1.00      8870
   macro avg       1.00      0.99      0.99      8870
weighted avg       1.00      1.00      1.00      8870



In [77]:
sentiment_analyzer = VS()

def sentiment_analysis(tweet):   
    sentiment = sentiment_analyzer.polarity_scores(tweet)    
    features = [sentiment['neg'], sentiment['pos'], sentiment['neu'], sentiment['compound']]
    #features = pandas.DataFrame(features)
    return features

def sentiment_analysis_array(tweets):
    features=[]
    for t in tweets:
        features.append(sentiment_analysis(t))
    return np.array(features)

In [87]:
X_train, X_val, y_train, y_val = train_test_split(trainset["tweet"], trainset["label"], test_size=0.33, random_state=42)
X_train, X_val, y_train, y_val=list(X_train), list(X_val), list(y_train), list(y_val)

In [88]:
X_train

['i believe that serena would not cheat yet even her manager has admitted to breaking the rules this was not her finest hour first a temper tantrum then calling the umpire a thief that crossed the line she is the one that should apologize',
 'is not  this bodyshaming  good to see the feminists and liberals have no issue with bodyshaming as long as it is directed towards a man',
 'funny how he believes antifa is a thing but calls himself reality checks',
 'sorry fella but connelly thought he was hollywood craigkardashian',
 'aw what that is devastating  hope you are ok   i love you biter',
 'because he is a liberal',
 'swear niggas make me wanna turn this phone off',
 'i did not know this  great job california   maga',
 'star tv',
 'of course you are  antiamerican and you wo not  leave  you will  stay here because as an antifa coward you can talk your nonsense with no fear of governmental reprisal  you have not  left yet and you never will',
 'if they picked a more suitable song it woul

In [89]:
# final_features = sentiment_analysis_array(tweets)
# final_test_features=sentiment_analysis_array(test_tweets)


train_sentiments=sentiment_analysis_array(X_train)
val_sentiments=sentiment_analysis_array(X_val)
# test_sentiments=sentiment_analysis_array(testset["tweet"])

In [90]:
print(train_sentiments.shape)
print(val_sentiments.shape)
# print(test_sentiments.shape)

(8870, 4)
(4370, 4)


In [91]:
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf=vectorizer.transform(X_val)
# X_test_tfidf=vectorizer.transform(list(testset["tweet"]))

In [92]:
X_train_tfidf = X_train_tfidf.toarray()

X_val_tfidf= X_val_tfidf.toarray()
# X_test_tfidf= X_test_tfidf.toarray()

In [93]:
x_train_sentiment=np.concatenate([X_train_tfidf,train_sentiments],axis=1)
x_val_sentiment=np.concatenate([X_val_tfidf,val_sentiments],axis=1)
# x_test_sentiment=np.concatenate([X_test_tfidf,test_sentiments],axis=1)


print(x_train_sentiment.shape)
print(x_val_sentiment.shape)
# print(x_test_sentiment.shape)

(8870, 15771)
(4370, 15771)


In [94]:
dict_sentiment_models={}
dict_sentiment_report={}

In [95]:
model = LogisticRegression().fit(x_train_sentiment,y_train)
y_preds = model.predict(x_val_sentiment)
report = classification_report( y_val, y_preds )
print(report)
acc=accuracy_score(y_val,y_preds)
print("Logistic Regression,Accuracy Score:" , acc)

              precision    recall  f1-score   support

         NOT       0.77      0.91      0.84      2903
         OFF       0.72      0.48      0.58      1467

    accuracy                           0.76      4370
   macro avg       0.75      0.69      0.71      4370
weighted avg       0.76      0.76      0.75      4370

Logistic Regression,Accuracy Score: 0.762700228832952


In [109]:
y_train_pred=model.predict(x_train_sentiment)
report = classification_report( y_train, y_train_pred )
print(report)

              precision    recall  f1-score   support

         NOT       0.81      0.96      0.88      5937
         OFF       0.86      0.55      0.67      2933

    accuracy                           0.82      8870
   macro avg       0.84      0.76      0.78      8870
weighted avg       0.83      0.82      0.81      8870



In [100]:
len(y_train)

8870

In [108]:
for i in range(len(y_train)):
    if y_train[i]==0:
        y_train[i]="OFF"
        
    else:
        y_train[i]="NOT"

In [104]:
from sklearn.model_selection import cross_validate
def cross_validation(model, _X, _y, _cv=3):
      '''Function to perform 5 Folds Cross-Validation
       Parameters
       ----------
      model: Python Class, default=None
              This is the machine learning algorithm to be used for training.
      _X: array
           This is the matrix of features.
      _y: array
           This is the target variable.
      _cv: int, default=5
          Determines the number of folds for cross-validation.
       Returns
       -------
       The function returns a dictionary containing the metrics 'accuracy', 'precision',
       'recall', 'f1' for both training set and validation set.
      '''
      _scoring = ['accuracy', 'precision', 'recall', 'f1_macro']
      results = cross_validate(estimator=model,
                               X=_X,
                               y=_y,
                               cv=_cv,
                               scoring=_scoring,
                               return_train_score=True)
      
      return results

In [105]:
print(cross_validation(model, x_train_sentiment, y_train))

{'fit_time': array([11.38905907, 10.67964721, 11.03497148]), 'score_time': array([0.21331906, 0.2031188 , 0.21007323]), 'test_accuracy': array([0.7456882 , 0.74940818, 0.75507442]), 'train_accuracy': array([0.82310164, 0.82513107, 0.82144065]), 'test_precision': array([0.75701718, 0.76074136, 0.76421053]), 'train_precision': array([0.81392842, 0.81252672, 0.80991029]), 'test_recall': array([0.91308742, 0.91258211, 0.91712986]), 'train_recall': array([0.95376453, 0.9603335 , 0.95805963]), 'test_f1_macro': array([0.67098592, 0.67753843, 0.6845089 ]), 'train_f1_macro': array([0.77723824, 0.77806536, 0.77326781])}


In [110]:
dict_sentiment_models["logistic"]=model
dict_sentiment_report["logistic"]=report

In [112]:
# rf=RandomForestClassifier()
# rf.fit(x_train_sentiment,y_train)
# y_preds = rf.predict(x_val_sentiment)
# acc1=accuracy_score(y_val,y_preds)
# report = classification_report( y_val, y_preds )
# print(report)
# print("Random Forest, Accuracy Score:",acc1)


KeyboardInterrupt



In [ ]:
# y_train_pred=model.predict(x_train_sentiment)
# report = classification_report( y_train, y_train_pred )
# print(report)

In [113]:
support =LinearSVC(random_state=20)
support.fit(x_train_sentiment,y_train)
y_preds = support.predict(x_val_sentiment)
acc3=accuracy_score(y_val,y_preds)
report = classification_report( y_val, y_preds )
print(report)

print("SVM, Accuracy Score:" , acc3)


dict_sentiment_models["svm"]=support
dict_sentiment_report["svm"]=report

              precision    recall  f1-score   support

         NOT       0.79      0.86      0.83      2903
         OFF       0.67      0.54      0.60      1467

    accuracy                           0.76      4370
   macro avg       0.73      0.70      0.71      4370
weighted avg       0.75      0.76      0.75      4370

SVM, Accuracy Score: 0.7565217391304347


In [114]:
y_train_pred=model.prefdict(x_train_sentiment)
report = classification_report( y_train, y_train_pred )
print(report)

              precision    recall  f1-score   support

         NOT       0.81      0.96      0.88      5937
         OFF       0.86      0.55      0.67      2933

    accuracy                           0.82      8870
   macro avg       0.84      0.76      0.78      8870
weighted avg       0.83      0.82      0.81      8870



In [115]:
model_pa = PassiveAggressiveClassifier(C = 0.6, random_state = 5)
model_pa.fit(x_train_sentiment,y_train)
y_preds = support.predict(x_val_sentiment)
acc3=accuracy_score(y_val,y_preds)
report = classification_report( y_val, y_preds )
print(report)
print("passive aggressive classifier accuracy:" , acc3)

dict_sentiment_models["pa"]=model_pa
dict_sentiment_report["pa"]=report

              precision    recall  f1-score   support

         NOT       0.79      0.86      0.83      2903
         OFF       0.67      0.54      0.60      1467

    accuracy                           0.76      4370
   macro avg       0.73      0.70      0.71      4370
weighted avg       0.75      0.76      0.75      4370

passive aggressive classifier accuracy: 0.7565217391304347


In [116]:
y_train_pred=model.predict(x_train_sentiment)
report = classification_report( y_train, y_train_pred )
print(report)

              precision    recall  f1-score   support

         NOT       0.81      0.96      0.88      5937
         OFF       0.86      0.55      0.67      2933

    accuracy                           0.82      8870
   macro avg       0.84      0.76      0.78      8870
weighted avg       0.83      0.82      0.81      8870



In [117]:
dict_sentiment_report

{'logistic': '              precision    recall  f1-score   support\n\n         NOT       0.81      0.96      0.88      5937\n         OFF       0.86      0.55      0.67      2933\n\n    accuracy                           0.82      8870\n   macro avg       0.84      0.76      0.78      8870\nweighted avg       0.83      0.82      0.81      8870\n',
 'svm': '              precision    recall  f1-score   support\n\n         NOT       0.79      0.86      0.83      2903\n         OFF       0.67      0.54      0.60      1467\n\n    accuracy                           0.76      4370\n   macro avg       0.73      0.70      0.71      4370\nweighted avg       0.75      0.76      0.75      4370\n',
 'pa': '              precision    recall  f1-score   support\n\n         NOT       0.79      0.86      0.83      2903\n         OFF       0.67      0.54      0.60      1467\n\n    accuracy                           0.76      4370\n   macro avg       0.73      0.70      0.71      4370\nweighted avg    

In [118]:
dict_sentiment_models

{'logistic': LogisticRegression(),
 'svm': LinearSVC(random_state=20),
 'pa': PassiveAggressiveClassifier(C=0.6, random_state=5)}

In [119]:
models_list=['logistic', 'svm', 'pa']

dict_outputs_sentiment={"test":{}, "val": {}}

In [121]:
for model in models_list:
    dict_outputs_sentiment["val"][model]=dict_sentiment_models[model].predict(x_val_sentiment)

In [122]:
# list_test_preds=np.array(list(dict_outputs_sentiment["test"].values()))
list_val_preds=np.array(list(dict_outputs_sentiment["val"].values()))

In [123]:
majority_val_prediction=majority_prediction(list_val_preds)
# majority_test_prediction=majority_prediction(list_test_preds)

In [124]:
print(classification_report(y_val, majority_val_prediction))

              precision    recall  f1-score   support

         NOT       0.79      0.87      0.83      2903
         OFF       0.67      0.54      0.60      1467

    accuracy                           0.76      4370
   macro avg       0.73      0.70      0.71      4370
weighted avg       0.75      0.76      0.75      4370



In [ ]:
# print(classification_report(testset["truelabel"], majority_test_prediction))

              precision    recall  f1-score   support

         NOT       0.83      0.93      0.88       620
         OFF       0.74      0.50      0.60       240

    accuracy                           0.81       860
   macro avg       0.78      0.72      0.74       860
weighted avg       0.80      0.81      0.80       860



In [125]:
X_train, X_val, y_train, y_val = train_test_split(trainset["tweet"], trainset["label"], test_size=0.33, random_state=42)
X_train, X_val, y_train, y_val=list(X_train), list(X_val), list(y_train), list(y_val)

In [126]:
train_sentiments=sentiment_analysis_array(X_train)
val_sentiments=sentiment_analysis_array(X_val)
# test_sentiments=sentiment_analysis_array(testset["tweet"])

In [127]:
trainval_sentiments=sentiment_analysis_array(trainset["tweet"])

In [128]:
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf=vectorizer.transform(X_val)
# X_test_tfidf=vectorizer.transform(list(testset["tweet"]))

In [129]:
X_trainval_tfidf=vectorizer.transform(list(trainset["tweet"]))

In [130]:
X_train_tfidf = X_train_tfidf.toarray()

X_val_tfidf= X_val_tfidf.toarray()
# X_test_tfidf= X_test_tfidf.toarray()


In [131]:
X_trainval_tfidf=X_trainval_tfidf.toarray()

In [132]:
train_data = []

for tweet in X_train:
  for i in sent_tokenize(tweet):
    temp=[]

    for j in word_tokenize(i):
      temp.append(j.lower())
    train_data.append(temp)

In [133]:
np.array(train_data).shape

C:\Users\udit1\AppData\Local\Temp\ipykernel_17912\2430969041.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(train_data).shape


(8870,)

In [134]:
# create doc2vec vector columns
# Initialize and train the model

#The input for a Doc2Vec model should be a list of TaggedDocument(['list','of','word'], [TAG_001]). 
#A good practice is using the indexes of sentences as the tags.
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(trainset["tweet"].apply(lambda x: x.split(" ")))]

In [135]:
# documents_test= [TaggedDocument(doc, [i]) for i, doc in enumerate(testset["tweet"].apply(lambda x: x.split(" ")))]

In [136]:
documents[5]

TaggedDocument(words=['liberals', 'are', 'all', 'kookoo'], tags=[5])

In [137]:
doc2vecmodel = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)


doc2vec_df = trainset["tweet"].apply(lambda x: doc2vecmodel.infer_vector(x.split(" "))).apply(pd.Series)
doc2vec_df.columns = ["doc2vec_vector_" + str(x) for x in doc2vec_df.columns]
doc2vec_df

,doc2vec_vector_0,doc2vec_vector_1,doc2vec_vector_2,doc2vec_vector_3,doc2vec_vector_4
0,0.102389,0.076852,0.002161,0.152961,0.050581
1,-0.032990,0.063043,0.119768,0.085978,-0.072681
2,0.675319,0.127533,0.693909,-0.545737,-0.316598
3,0.034878,0.065100,-0.001657,-0.115279,-0.132372
4,0.120925,0.103360,0.134708,0.118980,0.021685
...,...,...,...,...,...
13235,0.023730,0.023792,0.225468,-0.377667,0.003902
13236,0.119499,-0.010589,0.259035,0.005836,-0.042545
13237,-0.221585,0.126054,-0.048132,0.172517,-0.115095
13238,0.103950,0.120668,0.043100,-0.012822,0.011565


In [138]:
# doc2vec_df_test = testset["tweet"].apply(lambda x: doc2vecmodel.infer_vector(x.split(" "))).apply(pd.Series)
# doc2vec_df_test.columns = ["doc2vec_vector_" + str(x) for x in doc2vec_df_test.columns]
# doc2vec_df_test

In [139]:
# conctaenation of tf-idf scores, sentiment scores and doc2vec columns
train_doc2vec = np.concatenate([X_trainval_tfidf,trainval_sentiments,doc2vec_df],axis=1)
train_doc2vec.shape

(13240, 15776)

In [144]:
# test_doc2vec = np.concatenate([X_test_tfidf, test_sentiments,doc2vec_df_test],axis=1)
# test_doc2vec.shape

In [145]:
dict_doc2vec_models={}

dict_doc2vec_report={}

In [146]:

X = pd.DataFrame(train_doc2vec)
y = trainset["label"]
X_train_bow, X_test_bow, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.33)

support =LinearSVC(random_state=20)
support.fit(X_train_bow,y_train)
y_preds = support.predict(X_test_bow)
acc3=accuracy_score(y_test,y_preds)
report = classification_report( y_test, y_preds )
print(report)

print("SVM, Accuracy Score:" , acc3)

              precision    recall  f1-score   support

         NOT       0.79      0.86      0.83      2903
         OFF       0.67      0.55      0.60      1467

    accuracy                           0.76      4370
   macro avg       0.73      0.71      0.71      4370
weighted avg       0.75      0.76      0.75      4370

SVM, Accuracy Score: 0.7578947368421053


In [148]:
y_preds = support.predict(X_train_bow)
acc3=accuracy_score(y_train,y_preds)
report = classification_report(y_train, y_preds )
print(report)

print("SVM, Accuracy Score:" , acc3)

              precision    recall  f1-score   support

         NOT       0.96      0.98      0.97      5937
         OFF       0.97      0.93      0.95      2933

    accuracy                           0.96      8870
   macro avg       0.96      0.95      0.96      8870
weighted avg       0.96      0.96      0.96      8870

SVM, Accuracy Score: 0.9645997745208568


In [149]:
dict_doc2vec_models["svm"]=support

dict_doc2vec_report["svm"]=report

In [150]:
model = LogisticRegression().fit(X_train_bow,y_train)
y_preds = support.predict(X_test_bow)
acc3=accuracy_score(y_test,y_preds)
report = classification_report( y_test, y_preds )
print(report)

print("logistic Accuracy Score:" , acc3)

C:\Users\udit1\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

         NOT       0.79      0.86      0.83      2903
         OFF       0.67      0.55      0.60      1467

    accuracy                           0.76      4370
   macro avg       0.73      0.71      0.71      4370
weighted avg       0.75      0.76      0.75      4370

logistic Accuracy Score: 0.7578947368421053


In [152]:
y_preds = model.predict(X_train_bow)
acc3=accuracy_score(y_train,y_preds)
report = classification_report( y_train, y_preds )
print(report)

print("Logistic Regression:" , acc3)

              precision    recall  f1-score   support

         NOT       0.81      0.96      0.88      5937
         OFF       0.86      0.55      0.67      2933

    accuracy                           0.82      8870
   macro avg       0.84      0.75      0.78      8870
weighted avg       0.83      0.82      0.81      8870

Logistic Regression: 0.8225479143179256


In [153]:
dict_doc2vec_models["logistic"]=model

dict_doc2vec_report["logistic"]=report

In [154]:
model_pa = PassiveAggressiveClassifier(C = 0.6, random_state = 5)
model_pa.fit(X_train_bow,y_train)
y_preds = support.predict(X_test_bow)
acc3=accuracy_score(y_test,y_preds)
report = classification_report( y_test, y_preds )
print(report)

print("Passive Agressive Accuracy Score:" , acc3)

              precision    recall  f1-score   support

         NOT       0.79      0.86      0.83      2903
         OFF       0.67      0.55      0.60      1467

    accuracy                           0.76      4370
   macro avg       0.73      0.71      0.71      4370
weighted avg       0.75      0.76      0.75      4370

Passive Agressive Accuracy Score: 0.7578947368421053


In [156]:
y_preds = model_pa.predict(X_train_bow)
acc3=accuracy_score(y_train,y_preds)
report = classification_report( y_train, y_preds )
print(report)

print("Passive Agressive Accuracy Score:" , acc3)

              precision    recall  f1-score   support

         NOT       1.00      0.99      1.00      5937
         OFF       0.99      0.99      0.99      2933

    accuracy                           0.99      8870
   macro avg       0.99      0.99      0.99      8870
weighted avg       0.99      0.99      0.99      8870

Passive Agressive Accuracy Score: 0.9947012401352875


In [157]:
dict_doc2vec_models["pa"]=model_pa

dict_doc2vec_report["pa"]=report

In [158]:
models_list=['logistic', 'svm', 'pa']

dict_outputs_doc2vec={"test":{}, "val": {}}

In [160]:
for model in models_list:
  dict_outputs_doc2vec["val"][model]=dict_doc2vec_models[model].predict(X_test_bow)
#   dict_outputs_doc2vec["test"][model]=dict_doc2vec_models[model].predict(test_doc2vec)

In [161]:
# list_test_preds=np.array(list(dict_outputs_doc2vec["test"].values()))
list_val_preds=np.array(list(dict_outputs_doc2vec["val"].values()))

In [162]:
majority_val_prediction=majority_prediction(list_val_preds)
# majority_test_prediction=majority_prediction(list_test_preds)

In [164]:
print(classification_report(y_val, majority_val_prediction))

              precision    recall  f1-score   support

         NOT       0.79      0.86      0.83      2903
         OFF       0.67      0.55      0.60      1467

    accuracy                           0.76      4370
   macro avg       0.73      0.71      0.71      4370
weighted avg       0.75      0.76      0.75      4370



Using only doc2vec and sentiment features

In [166]:
# conctaenation of tf-idf scores, sentiment scores and doc2vec columns
train_doc2vec = np.concatenate([trainval_sentiments,doc2vec_df],axis=1)
train_doc2vec.shape

(13240, 9)

In [167]:

X = pd.DataFrame(train_doc2vec)
y = trainset["label"]
X_train_bow, X_test_bow, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.33)

support =LinearSVC(random_state=20)
support.fit(X_train_bow,y_train)
y_preds = support.predict(X_test_bow)
acc3=accuracy_score(y_test,y_preds)
report = classification_report( y_test, y_preds )
print(report)

print("SVM, Accuracy Score:" , acc3)

              precision    recall  f1-score   support

         NOT       0.73      0.89      0.81      2903
         OFF       0.63      0.36      0.46      1467

    accuracy                           0.71      4370
   macro avg       0.68      0.63      0.63      4370
weighted avg       0.70      0.71      0.69      4370

SVM, Accuracy Score: 0.7144164759725401


In [168]:
model = LogisticRegression().fit(X_train_bow,y_train)
y_preds = support.predict(X_test_bow)
acc3=accuracy_score(y_test,y_preds)
report = classification_report( y_test, y_preds )
print(report)

print("logistic Accuracy Score:" , acc3)

              precision    recall  f1-score   support

         NOT       0.73      0.89      0.81      2903
         OFF       0.63      0.36      0.46      1467

    accuracy                           0.71      4370
   macro avg       0.68      0.63      0.63      4370
weighted avg       0.70      0.71      0.69      4370

logistic Accuracy Score: 0.7144164759725401
